Not archiving the data right now, this exercise results in a pretty easy way to improve ticdat.

In [1]:
import ticdat
import time

In [2]:
pandat = ticdat.PanDatFactory(table_data = [[], ["SoldCount", "StrengthFactor", "ItemCount"]])

In [3]:
pandat.set_data_type("table_data", "SoldCount", min=0, max=50, inclusive_min=True, 
                     inclusive_max=True, nullable=False)

In [4]:
dat = pandat.csv.create_pan_dat("pandat_data")

In [5]:
dat.table_data["SoldCount"].max(), dat.table_data["SoldCount"].min()

(73.0, 0.0)

In [6]:
start_time = time.time()

In [7]:
all_fails = pandat.find_data_type_failures(dat)

In [8]:
df_with_bad_rows = all_fails['table_data', "SoldCount"]

In [9]:
len(dat.table_data)

198917

In [10]:
len(df_with_bad_rows)

122924

In [11]:
print(f"Time to do bulk query using apply under the hood in sec {time.time()-start_time}")

Time to do bulk query using apply under the hood in sec 2.3052430152893066


In [12]:
len(dat.table_data.columns)

14

2.5 seconds is slow compared to the bulk copy.

In [13]:
%time dat.table_data.copy(deep=True)

CPU times: user 50.5 ms, sys: 25.6 ms, total: 76.2 ms
Wall time: 13 ms


,SoldCount,StrengthFactor,ItemCount,Order,File_Type,SKU_number,SoldFlag,MarketingType,ReleaseNumber,New_Release_Flag,PriceReg,ReleaseYear,LowUserPrice,LowNetPrice
0,0.0,6.827430e+05,8,2,Historical,1737127,0.0,D,15,1,44.99,2015,28.97,31.84
1,0.0,1.016014e+06,39,3,Historical,3255963,0.0,D,7,1,24.81,2005,0.00,15.54
2,0.0,3.404640e+05,34,4,Historical,612701,0.0,D,0,0,46.00,2013,30.19,27.97
3,1.0,3.340110e+05,20,6,Historical,115883,1.0,D,4,1,100.00,2006,133.93,83.15
4,1.0,1.287938e+06,28,7,Historical,863939,1.0,D,2,1,121.95,2010,4.00,23.99
5,0.0,1.783153e+06,33,8,Historical,214948,0.0,D,0,0,132.00,2011,138.98,13.64
6,0.0,2.314801e+06,33,9,Historical,484059,0.0,D,13,1,95.95,2010,90.77,46.49
7,0.0,7.211110e+05,57,10,Historical,146401,0.0,D,4,1,207.80,2011,187.90,30.56
8,0.0,4.366670e+05,36,11,Historical,110568,0.0,D,11,1,119.81,2008,63.99,96.92
9,0.0,6.652211e+06,19,12,Historical,764270,0.0,D,5,1,49.95,2004,53.99,28.99


In [14]:
start_time = time.time()

In [15]:
def good_row(row):
    val = row[0]
    try:
        if 0 <= val <= 50:
            return True
    except:
        return False


In [16]:
bad_rows_by_tuples = [not good_row(row) for row in dat.table_data.itertuples(index=False)]

In [17]:
print(f"Time to do bulk query using apply under the hood in sec {time.time()-start_time}")

Time to do bulk query using apply under the hood in sec 0.20337796211242676


In [18]:
len([x for x in bad_rows_by_tuples if x])

122924

In [19]:
badded = dat.table_data[bad_rows_by_tuples]

In [20]:
len(badded)

122924